# Machine Translation with Bidirectional LSTM, without Attention
Adapted from Natural Language Processing Demystified
https://nlpdemystified.org<br>



**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
import io
import json
import numpy as np
import pandas as pd
import random
import re
import tensorflow as tf
import unicodedata
!pip install sacrebleu
from sacrebleu.metrics import BLEU

from google.colab import files
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

# CORPUS PREPROCESSING

Our first model will use one BiLSTM encoder and one simple LSTM decoder, to translate Hungarian to English. We're going with Hungarian here because it's a particularly challenging language to tackle with few resources.
<br><br>
The dataset we'll use comes from **Tatoeba**, a collection of sentence translations in a variety of languages sourced from volunteers:
<br>
https://tatoeba.org/en

Download the training set and load its contents

In [ ]:
# Download the training, validation, and test sets.
!wget https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt
!wget https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_val.txt
!wget https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_test.txt

In [ ]:
with open('hun_eng_pairs_train.txt') as file:
  train = [line.rstrip() for line in file]
with open('hun_eng_pairs_val.txt') as file:
  val = [line.rstrip() for line in file]
with open('hun_eng_pairs_test.txt') as file:
  test = [line.rstrip() for line in file]

Each entry consists of a Hungarian sentence followed by its English translation, separated by a **\<sep\>** delimiter

In [ ]:
train[:3]

This is a relatively tiny dataset for neural machine translation, but we'll see how far we can get.

In [ ]:
print("Training set size: " + str(len(train)))
print("Validation set size: " + str(len(val)))
print("Test set size: " + str(len(test)))

In [ ]:
# Separate the input (Hungarian) and target (English) sentences into separate lists.
SEPARATOR = '<sep>'
train_input, train_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))
val_input, val_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))
test_input, test_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))

In [ ]:
print(train_input[:3])
print(train_target[:3])

Since we're dealing with a source language that uses accented characters, it's important to apply *Unicode normalization*
<br><br>
In the example below, two different sets of Unicode yield the same character visually. The first Unicode is for an accented 'a', while the second Unicode combines an 'a' with an accent mark.

In [ ]:
print("\u00E1", "\u0061\u0301")

Though these characters look the same to us reading them, they'll be treated differently by a model. So to avoid this, the following function normalizes any accented characters into the same set of Unicode, and then replaces them with their ASCII equivalents.<br>
https://docs.python.org/3/library/unicodedata.html
<br><br>
Here's an informative article on the importance of Unicode normalization and how to go about it (including what _NFD_ means):<br>
https://towardsdatascience.com/what-on-earth-is-unicode-normalization-56c005c55ad0

In [ ]:
# Unicode normalization
def normalize_unicode(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        #if unicodedata.category(c) != 'Mn'
        )

We're building a **word**-based translation model, but we still want to keep punctuation and treat them as separate tokens, so we'll insert a space between any relevant punctuation and the characters around them. This way, our tokenizer (which won't filter out punctuation) will output punctuation marks as separate tokens.

This function does that and unicode normalization.

In [ ]:
def preprocess_sentence(s):
  s = normalize_unicode(s)
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  return s

In [ ]:
# Preprocess both the source and target sentences
train_preprocessed_input = [preprocess_sentence(s) for s in train_input]
train_preprocessed_target = [preprocess_sentence(s) for s in train_target]

After preprocessing, the unicode should be normalized and there should be spaces on either side of any punctuation.

In [ ]:
train_preprocessed_input[:3]

Like the language model we built in the [previous demo](https://github.com/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb), we'll use **Teacher Forcing** with our translation model (specifically, the decoder). This begins by placing a  start-of-sentence tag (_\<sos\>_) and end-of-sentence tag (_\<eos\>_) at the beginning and end of each target sentence, respectively.

In [ ]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [ ]:
train_tagged_preprocessed_target = tag_target_sentences(train_preprocessed_target)

In [ ]:
train_tagged_preprocessed_target[:3]

Next, we'll tokenize our input and target sentences, taking care to keep relevant punctuation.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
<br><br>
Note that we're also including an out-of-vocabulary token (_\<unk\>_) in the tokenizer initialization. At inference time, if the tokenizer encounters a word it didn't see during the initial fit on the training data, that word will be replaced with _\<unk\>_ and the translation system will need to cope with it.

In [ ]:
# Tokenizer for the Hungarian input sentences. Note how we're not filtering punctuation.
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
source_tokenizer.fit_on_texts(train_preprocessed_input)
unescapedWordCountDict = {}

In [ ]:
source_vocab_size = len(source_tokenizer.word_index) + 1
print(source_vocab_size)

In [ ]:
# Tokenizer for the English target sentences.
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
target_tokenizer.fit_on_texts(train_tagged_preprocessed_target)
target_tokenizer.get_config()

In [ ]:
target_vocab_size = len(target_tokenizer.word_index) + 1
print(target_vocab_size)

Next, we'll vectorize the input and target sentences just like we did in the last few demos.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [ ]:
train_encoder_inputs = source_tokenizer.texts_to_sequences(train_preprocessed_input)

In [ ]:
print(train_encoder_inputs[:3])
print(source_tokenizer.sequences_to_texts(train_encoder_inputs[:3]))

For teacher forcing, we'll create two copies of each vectorized **target** sentence, with the second copy shifted over by one.

The function below takes a collection of sentences, vectorizes them, then returns two copies of each. The first will include every token except the last, the second will include every token except the first.

In [ ]:
def generate_decoder_inputs_targets(sentences, tokenizer):
  seqs = tokenizer.texts_to_sequences(sentences)
  decoder_inputs = [s[:-1] for s in seqs] # Drop the last token in the sentence.
  decoder_targets = [s[1:] for s in seqs] # Drop the first token in the sentence.

  return decoder_inputs, decoder_targets

In [ ]:
train_decoder_inputs, train_decoder_targets = generate_decoder_inputs_targets(train_tagged_preprocessed_target,
                                                                              target_tokenizer)

Each token of a *decoder_input* sentence will be fed to the decoder as the **next** expected token, and each token of a *decoder_target* sentence will be used to calculate the loss against the decoder's actual output; exactly as we covered in the slides.

In [ ]:
print(train_decoder_inputs[0], train_decoder_targets[0])

print(target_tokenizer.sequences_to_texts(train_decoder_inputs[:1]),
      target_tokenizer.sequences_to_texts(train_decoder_targets[:1]))

Exactly as we did in the previous [RNN demo](https://github.com/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb), we'll pad all sequences to a uniform length.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
max_encoding_len = len(max(train_encoder_inputs, key=len))
max_encoding_len

In [ ]:
max_decoding_len = len(max(train_decoder_inputs, key=len))
max_decoding_len

In [ ]:
padded_train_encoder_inputs = pad_sequences(train_encoder_inputs, max_encoding_len, padding='post', truncating='post')
padded_train_decoder_inputs = pad_sequences(train_decoder_inputs, max_decoding_len, padding='post', truncating='post')
padded_train_decoder_targets = pad_sequences(train_decoder_targets, max_decoding_len, padding='post', truncating='post')

In [ ]:
print(padded_train_encoder_inputs[0])
print(padded_train_decoder_inputs[0])
print(padded_train_decoder_targets[0])

When converting a padded sequence back to text, padding is replaced with the out-of-vocabulary *\<unk\>* token.

In [ ]:
target_tokenizer.sequences_to_texts([padded_train_decoder_inputs[0]])

The training dataset is now ready, and we can follow the same preprocessing steps to prepare the validation dataset.

In [ ]:
def process_dataset(dataset):

  # Split the French and English sentences into separate lists.
  input, output = map(list, zip(*[pair.split(SEPARATOR) for pair in dataset]))

  # Unicode normalization and inserting spaces around punctuation.
  preprocessed_input = [preprocess_sentence(s) for s in input]
  preprocessed_output = [preprocess_sentence(s) for s in output]

  # Tag target sentences with <sos> and <eos> tokens.
  tagged_preprocessed_output = tag_target_sentences(preprocessed_output)

  # Vectorize encoder source sentences.
  encoder_inputs = source_tokenizer.texts_to_sequences(preprocessed_input)

  # Vectorize and create decoder input and target sentences.
  decoder_inputs, decoder_targets = generate_decoder_inputs_targets(tagged_preprocessed_output,
                                                                    target_tokenizer)

  # Pad all collections.
  padded_encoder_inputs = pad_sequences(encoder_inputs, max_encoding_len, padding='post', truncating='post')
  padded_decoder_inputs = pad_sequences(decoder_inputs, max_decoding_len, padding='post', truncating='post')
  padded_decoder_targets = pad_sequences(decoder_targets, max_decoding_len, padding='post', truncating='post')

  return padded_encoder_inputs, padded_decoder_inputs, padded_decoder_targets


In [ ]:
# Process validation dataset
padded_val_encoder_inputs, padded_val_decoder_inputs, padded_val_decoder_targets = process_dataset(val)
# Process test dataset
padded_test_encoder_inputs, padded_test_decoder_inputs, padded_test_decoder_targets = process_dataset(test)

# BUILDING THE MODEL



In [ ]:
# Hyperparameters

embedding_dim = 128
hidden_dim = 256
default_dropout=0.2
batch_size = 32
epochs = 12

Unlike the models we've built up to this point, this model will have **two** inputs:<br>
1. The encoder receives the source sentences (Hungarian) and generates the initial state inputs for the decoder.
2. The decoder receives the decoder input sentences (English) for teacher forcing.
<br>

Since that's the case, we won't use the Keras **Sequential API**. Rather, we'll use the **Functional API** which will give us more flexibility in expressing our model layers as graphs. Fortunately, the work remains pretty intuitive as long as we keep in mind what's happening behind the scenes.
<br>

https://keras.io/api/models/sequential/<br>
https://www.tensorflow.org/guide/keras/functional/

The code in the following cell specifies the encoder. If you haven't used the **Functional API** before, the most important thing to realize here is this: Each layer is like a _node_ in a graph. Each time we pass the output of one layer to the next, it creates a link between the two _nodes_ and allows data to flow through. Once the whole training model is built, we'll visualize it for greater clarity.

In [ ]:
from keras.layers import Concatenate

# The initial encoder input layer which will take in padded sequences. We're specifying
# a shape of None here but you can specify it upfront if you want since we
# know what the max encoding length is.
encoder_inputs = layers.Input(shape=[None], name='encoder_inputs')

# The embedding layer. Similar to what we did in the RNN demo.
encoder_embeddings = layers.Embedding(source_vocab_size,
                                      embedding_dim,
                                      mask_zero=True,
                                      name='encoder_embeddings')

# Passing the input layer output to the embedding layer creates a link between the
# two. Input sequences will now flow into the embedding layer which will output
# a sequence of embeddings.
encoder_embedding_output = encoder_embeddings(encoder_inputs)


# We're not using any kind of attention mechanism in this model, so setting only
# return_state to True is enough. return_sequences remains False.
encoder_lstm = layers.Bidirectional(layers.LSTM(hidden_dim,
                           return_state=True,
                           dropout=default_dropout,
                           name='encoder_lstm'), name="bidirectional")

# The BiLSTM encoder returns five values:
# - the encoder output, as the concatenation of the outputs from each direction
# - the hidden state from the forward direction
# - the cell state vector from the forward direction
# - the hidden state from the backward direction
# - the cell state vector from the backward direction
encoder_outputs, state_h, state_c, b_state_h, b_state_c = encoder_lstm(encoder_embedding_output)

# We concatenate the forward and backward state vectors before passing them to the decoder
state_h = Concatenate()([state_h, b_state_h])
state_c = Concatenate()([state_c, b_state_c])

# The final hidden and cell states from the encoder will be the the
# initial states for the decoder.
encoder_states = (state_h, state_c)

The code for specifying the decoder is similar except for three additions:
1. The LSTM has *return_sequences* set to *True* since we'll need the hidden state outputted at each timestep (similar to our PoS tagger and language model from the [RNN demo](https://github.com/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb)).

2. The decoder's LSTM takes an *initial_state*, the value for which is from the encoder.

3. There's a *softmax* layer in the end to generate a probability distribution over the target (English) vocabulary.

In [ ]:
decoder_inputs = layers.Input(shape=[None], name='decoder_inputs')


decoder_embeddings = layers.Embedding(target_vocab_size,
                                      embedding_dim,
                                      mask_zero=True,
                                      name='decoder_embeddings')


decoder_embedding_output = decoder_embeddings(decoder_inputs)

# As the forward and backward states from the encoder were concatenated,
# the decoder hidden dimension has to be double the size.
# The decoder is a regular one-way LSTM (BiLSTM decoders are very rare).
decoder_lstm = layers.LSTM(2 * hidden_dim,
                           return_sequences=True,
                           return_state=True,
                           dropout=default_dropout,
                           name='decoder_lstm')


# Set the decoder's initial state to the encoder's final output states. Since
# return_sequences is set to True, decoder_outputs is going to be a collection of
# the decoder's hidden state at each timestep. Also note that since we don't need
# the decoder's final hidden output and cell states, those are just set to _.
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_output, initial_state=encoder_states)

# Have a softmax layer in the end to create a probability distribution for the output word.
decoder_dense = layers.Dense(target_vocab_size, activation='softmax', name='decoder_dense')

# The probability distribution for the output word.
y_proba = decoder_dense(decoder_outputs)

We can now create our model and specify that it has **two** inputs and one output.
<br><br>
Note that *accuracy* is a crude metric for translation models and just serves as a proxy for how our model is doing. As we covered in the slides, the popular performance metric is BLEU.

In [ ]:
# Note how the model is taking two inputs in an array.
model = tf.keras.Model([encoder_inputs, decoder_inputs], y_proba, name='hu_en_mt_bilstm_no_attention')

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics='sparse_categorical_accuracy')
model.summary()

We can visualize our model to get a better idea of the flow we've built.

I personally found keeping the right matrix dimensions in mind more challenging and tricky than the theoretical concepts. So here are the dimensions we're working with in this training model.

In [ ]:
print('encoder_inputs layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len), (batch_size, max_encoding_len)))
print()
print('encoder_embeddings layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len), (batch_size, max_encoding_len, embedding_dim)))
print()
print('encoder_lstm layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len, embedding_dim), [(batch_size, hidden_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)]))
print()
print()
print('decoder_inputs layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len), (batch_size, max_decoding_len)))
print()
print('decoder_embeddings layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len), (batch_size, max_decoding_len, embedding_dim)))
print()
print('decoder_lstm layer\n input dimension {}\n output dimension: {}'.format([(batch_size, max_decoding_len, embedding_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)], [(batch_size, max_decoding_len, hidden_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)]))
print()
print('decoder_dense layer(softmax)\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len, hidden_dim), (batch_size, max_decoding_len, target_vocab_size)))

# TRAINING AND SAVING RESULTS




Because this model takes a while to train, we're using model checkpoints to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.
https://keras.io/api/callbacks/model_checkpoint/

In [ ]:
# Saving this to a folder on my local machine.
filepath="./HuEnMTBiLSTMNoAttention/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

We'll also use **early stopping** as we have recently.
<br><br>
The *fit* method below is commmented out because I trained the model ahead of time and saved it. I also saved the tokenizers.
<br><br>
If you want to train it yourself, feel free to uncomment and execute it. Keep in mind that because of the random weight initialization, your trained model's output will likely differ from mine.

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#history = model.fit([padded_train_encoder_inputs, padded_train_decoder_inputs], padded_train_decoder_targets,
#                     batch_size=batch_size,
#                     epochs=12,
#                     validation_data=([padded_val_encoder_inputs, padded_val_decoder_inputs], padded_val_decoder_targets),
#                     callbacks=[cp_callback, es_callback])

If you choose to train your own model, the following functions save the model and tokenizers.

In [ ]:
###### Save the model.
#model.save('hu_en_mt_bilstm_no_attention')

###### Save the tokenizers as JSON files. The resulting files can be downloaded by left-clicking on them.
source_tokenizer_json = source_tokenizer.to_json()
with io.open('source_tokenizer.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(source_tokenizer_json, ensure_ascii=False))

target_tokenizer_json = target_tokenizer.to_json()
with io.open('target_tokenizer.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(target_tokenizer_json, ensure_ascii=False))

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

###### Zip and download the model to your Google Drive account.
#!zip -r ./hu_en_mt_no_attention.zip ./hu_en_mt_no_attention
#!cp ./hu_en_mt_no_attention.zip /content/drive/MyDrive/Colab\ Notebooks

At this point, we can retrieve the saved model and saved tokenizers and try them on the **test** dataset.

In [ ]:
# Retrieve the tokenizers.
#!wget https://github.com/futuremojo/nlp-demystified/raw/main/models/nmt_no_attention/hun_eng_s2s_nmt_no_attention_tokenizers.zip
#!unzip -o hun_eng_s2s_nmt_no_attention_tokenizers.zip

The tokenizers can be loaded using the *tokenizer_from_json* method.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/tokenizer_from_json

In [ ]:
#with open('source_tokenizer.json') as f:
#   data = json.load(f)
#    source_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

#with open('target_tokenizer.json') as f:
#    data = json.load(f)
#    target_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

In [ ]:
# Retrieve the model.
!wget !wget https://partage.imt.fr/index.php/s/9zjHTGn6ifC9W5G/download/hu_en_mt_bilstm_no_attention.zip
!unzip -o hu_en_mt_bilstm_no_attention.zip

In [ ]:
# Load the model.
model = tf.keras.models.load_model('hu_en_mt_bilstm_no_attention')

# SIMPLE EVALUATION

The *test* dataset contains sentences (and most certainly words) unseen by the model.

In [ ]:
# Evaluate the model on the test set.
# It is a simple accuracy-based evaluation, not ideal for machine translation output.
# We will also compute BLEU scores below, which are still not ideal, but still a standard evaluation measure for MT.
model.evaluate([padded_test_encoder_inputs, padded_test_decoder_inputs], padded_test_decoder_targets)

# RECREATING AN ENCODER AND A DECODER FOR INFERENCE

Our training model uses teacher forcing, but that won't be the case for inference. So we'll take the trained layers from our training model, and create **separate**, stand-alone encoder and decoder models. This will give us much greater control over how the output is created as we'll see. (e.g. feeding the current output as the input to the next time step).

In [ ]:
# These are the layers of our trained model.
[layer.name for layer in model.layers]

Creating a stand-alone encoder is just a matter of retrieving the trained layers by name and re-creating the graph.

The major difference here is the last line where we're calling *tf.keras.Model* to create a stand-alone encoder with *encoder_inputs* as the input and *encoder_states* as the output.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#get_layer

In [ ]:
encoder_inputs = model.get_layer('encoder_inputs').input

encoder_embedding_layer = model.get_layer('encoder_embeddings')
encoder_embeddings = encoder_embedding_layer(encoder_inputs)

encoder_lstm = model.get_layer('bidirectional')

_, encoder_state_h, encoder_state_c, b_state_h, b_state_c = encoder_lstm(encoder_embeddings)

encoder_state_h = Concatenate()([encoder_state_h, b_state_h])
encoder_state_c = Concatenate()([encoder_state_c, b_state_c])

encoder_states = [encoder_state_h, encoder_state_c]

# Our stand-alone encoder model. encoder_inputs is the input to the encoder,
# and encoder_states is the expected output.
encoder_model_no_attention = tf.keras.Model(encoder_inputs, encoder_states)

Creating the decoder is similar. The major difference here is that there are two additional inputs to the decoder representing its LSTM hidden state (*decoder_input_state_h*) and cell state (*decoder_input_state_c*), respectively.
<br><br>
They are there because the encoder and decoder are now separate models, so we'll manually take the encoder's output (its final states) and use them as the decoder's initial state. From there, at each time step, we'll take the decoder's state outputs and feed them to the next time step.
<br><br>
The stand-alone decoder now takes as input:
- Either the '\<sos\>' token (at the beginning) or the output from the previous timestep. Either way, it'll now take only one token at a time.
- Hidden and cell states, either from the encoder initially or from the decoder's last time step.
<br>

And it outputs a probability distribution for the current output, and new hidden and cell states.

In [ ]:
decoder_inputs = model.get_layer('decoder_inputs').input

decoder_embedding_layer = model.get_layer('decoder_embeddings')
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

# Inputs to represent the decoder's LSTM hidden and cell states. We'll populate
# these manually using the encoder's output for the initial state.
decoder_input_state_h = tf.keras.Input(shape=(2 * hidden_dim,), name='decoder_input_state_h')
decoder_input_state_c = tf.keras.Input(shape=(2 * hidden_dim,), name='decoder_input_state_c')
decoder_input_states = [decoder_input_state_h, decoder_input_state_c]

decoder_lstm = model.get_layer('decoder_lstm')

decoder_sequence_outputs, decoder_output_state_h, decoder_output_state_c = decoder_lstm(
    decoder_embeddings, initial_state=decoder_input_states
)

# Update hidden and cell states for the next time step.
decoder_output_states = [decoder_output_state_h, decoder_output_state_c]

decoder_dense = model.get_layer('decoder_dense')
y_proba = decoder_dense(decoder_sequence_outputs)

decoder_model_no_attention = tf.keras.Model(
    [decoder_inputs] + decoder_input_states,
    [y_proba] + decoder_output_states
)

# USING THE MACHINE TRANSLATOR

The method below translates a sentence from the source language to the target language. It encodes the source sentence as usual, then feeds the encoder's state outputs and the \<sos\> token into the decoder.
<br><br>
The decoder's outputs (the resulting word and its hidden/cell states) are then fed back to the decoder at the next time step. This continues until either a max word limit is reached or an \<eos\> token is generated.

In [ ]:
def translate_without_attention(sentence: str,
                                source_tokenizer, encoder,
                                target_tokenizer, decoder,
                                max_translated_len = 30):

  # Vectorize the source sentence and run it through the encoder.
  input_seq = source_tokenizer.texts_to_sequences([sentence])

  # Get the tokenized sentence to see if there are any unknown tokens.
  tokenized_sentence = source_tokenizer.sequences_to_texts(input_seq)

  states = encoder.predict(input_seq)

  current_word = '<sos>'
  decoded_sentence = []

  while len(decoded_sentence) < max_translated_len:

    # Set the next input word for the decoder.
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_tokenizer.word_index[current_word]

    # Determine the next word.
    target_y_proba, h, c = decoder.predict([target_seq] + states)
    target_token_index = np.argmax(target_y_proba[0, -1, :])
    current_word = target_tokenizer.index_word[target_token_index]

    if (current_word == '<eos>'):
      break

    decoded_sentence.append(current_word)
    states = [h, c]

  return tokenized_sentence[0], ' '.join(decoded_sentence)


To test it out, we'll sample a bunch of sentences from the *test* dataset and translate them.

In [ ]:
# random.seed is just here to re-create results.
random.seed(1)
sentences = random.sample(test, 15)
sentences

In [ ]:
def translate_sentences(sentences, translation_func, source_tokenizer, encoder,
                        target_tokenizer, decoder):
  translations = {'Tokenized Original': [], 'Reference': [], 'Translation': []}

  for s in sentences:
    source, target = s.split(SEPARATOR)
    source = preprocess_sentence(source)
    tokenized_sentence, translated = translation_func(source, source_tokenizer, encoder,
                                                      target_tokenizer, decoder)

    translations['Tokenized Original'].append(tokenized_sentence)
    translations['Reference'].append(target)
    translations['Translation'].append(translated)

  return translations

We'll load the results into a Pandas **DataFrame** for easier viewing.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [ ]:
translations_no_attention = pd.DataFrame(translate_sentences(sentences, translate_without_attention,
                                                             source_tokenizer, encoder_model_no_attention,
                                                             target_tokenizer, decoder_model_no_attention))
translations_no_attention

So performance here tends to be mixed with a few translations which are off, a few which match the reference exactly, and a few interesting ones where the translation doesn't match the reference but conveys the same idea. But overall, pretty good.
<br><br>
There are several limitations we're working with:
- Hungarian is a difficult language because of its flexible structure and high expressiveness. The conjugation style leads to several English words often mapping to one word in Hungarian. Even Google Translate tends to have a high error rate with Hungarian.
- This is a small dataset and to build something robust requires much more data. It also isn't very wide and diverse.
- We're using a simple model with one LSTM for the encoder, and one LSTM for the decoder. In contrast, when Google Translate was still fully recurrence-based, it used an eight-layer LSTM for the encoder, and an eight-layer LSTM for the decoder (though in this case, I still think the data is the primary limitation).
- Translation remains a tricky and hard problem. For examples of issues that arise in translation, check out the video from Rasa in the *References* section at the end of the notebook.
<br><br>
Still, we managed to get some decent and interesting results with a simple approach.

In addition, let's take a look at results of translating the longest sentences in the training set to see how good they are.

In [ ]:
pairs = train.copy()
pairs.sort(key=lambda s: len(s))
longer_sentences = pairs[-10:]
longer_sentences

In [ ]:
longer_translations_wo_attention = pd.DataFrame(translate_sentences(longer_sentences, translate_without_attention,
                                                                      source_tokenizer, encoder_model_no_attention,
                                                                      target_tokenizer, decoder_model_no_attention))
longer_translations_wo_attention

In [ ]:
bleu = BLEU(max_ngram_order=3)
shortBleu = bleu.corpus_score(translations_no_attention['Translation'].tolist(), [translations_no_attention['Reference'].tolist()])
longBleu  = bleu.corpus_score(longer_translations_wo_attention['Translation'].tolist(), [longer_translations_wo_attention['Reference'].tolist()])
print("BLEU on short sentences: " + str(shortBleu))
print("BLEU on long sentences:  " + str(longBleu))


# References

Challenges in machine tanslation from RASA:
https://www.youtube.com/watch?v=94K4LU_Pe8Y

Sequence to Sequence Learning with Neural Networks

https://arxiv.org/abs/1409.3215


Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
https://arxiv.org/abs/1406.1078


Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation
- 8 encoder and decoder layers
- Uses attention mechanism

 https://arxiv.org/abs/1609.08144




Neural Machine Translation by Jointly Learning to Align and Translate
 https://arxiv.org/abs/1409.0473


Effective Approaches to Attention-based Neural Machine Translation
https://arxiv.org/abs/1508.04025
